In [55]:
import datetime
from functools import lru_cache
import numpy as np
from scipy.stats.mstats import gmean
import matplotlib.pyplot as plt
import yfinance as yf

np.random.seed(1123)

In [26]:
# Monte Carlo Simulation
spx_payoff = np.array([0.85, 1.0, 1.15, 1.30, 1.55])
spx_probs = np.array([11/120, 22/120, 30/120, 37/120, 20/120])
n_trials = 10_000
n_rolls = 25
mc = np.random.choice(spx_payoff, (n_trials, n_rolls), p=spx_probs)

In [69]:
@lru_cache(maxsize=None)
def get_sp500_yearly_returns(end_year, start_year=1930):
    # Initialize a yfinance session
    yf.pdr_override()

    # Define the ticker symbol for the S&P 500
    symbol = '^GSPC'

    # Initialize a dictionary to store YOY returns for each year
    yearly_returns = {}

    # Loop through each year and calculate YOY returns
    for year in range(start_year, end_year + 1):
        start_date = datetime.datetime(year, 1, 1)
        end_date = datetime.datetime(year + 1, 1, 1) - datetime.timedelta(days=1)

        # Fetch historical data for the S&P 500 for the current year
        sp500 = yf.download(symbol, start=start_date, end=end_date, progress=False)

        # Calculate the YOY return for the current year
        closing_price_start = sp500['Adj Close'].iloc[0]
        closing_price_end = sp500['Adj Close'].iloc[-1]

        year_over_year_return = (closing_price_end - closing_price_start) / closing_price_start * 100

        # Store the YOY return in the dictionary
        yearly_returns[year] = year_over_year_return

    return yearly_returns

In [70]:
yearly_returns = get_sp500_yearly_returns(end_year=2022)